# EcoInnovators Round 2 - YOLOv12m-seg Solar Panel Segmentation Training

**Objective**: Train a state-of-the-art instance segmentation model for rooftop solar panel detection.

**Model**: YOLOv12m-seg (latest, greatest)

**Training Time Budget**: ~4 hours on H200

**Key Improvements over Round 1**:
1. Segmentation instead of detection (accurate area calculation)
2. Multiple high-quality datasets merged (~10k+ images)
3. Negative samples for false positive reduction
4. Enhanced augmentation for robustness

---

## Dataset Strategy

We merge multiple datasets to create a robust, diverse training set:

| Priority | Dataset | Source | Images | Purpose |
|----------|---------|--------|--------|--------|
| **1** | Solar panels seg | Roboflow (RUT) | ~4,010 | Primary - Large scale instance segmentation |
| **2** | NL Solar Panel Seg | Roboflow | ~4,160 | European aerial imagery diversity |
| **3** | Solar Panels - Polygons | Roboflow | 86 | High-quality polygon annotations |
| **4** | Solar panels RF100 | Roboflow 100 | ~1,000 | Benchmark quality |
| **5** | Solar PV Detection | Roboflow | ~500 | Additional variety |
| **6** | WW Solar Panel | Roboflow | ~500 | Global coverage |

**Total: ~10,000+ diverse segmentation samples**

## 1. Environment Setup

In [ ]:
import os
import torch

print("="*80)
print("GPU VERIFICATION")
print("="*80)
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU Memory: {gpu_mem:.2f} GB")
    
    if gpu_mem > 70:
        print("\n✓ H200/H100 detected - Optimal for YOLOv12m-seg training!")
    elif gpu_mem > 40:
        print("\n✓ A100 detected - Great for training!")
else:
    print("⚠ WARNING: No GPU detected. Training will be very slow.")

print("\n📦 Installing dependencies...")
!pip install -q ultralytics>=8.3.0
!pip install -q roboflow
!pip install -q opencv-python shapely pandas openpyxl matplotlib
!pip install -q mlflow

print("✓ Dependencies installed!")

In [ ]:
from ultralytics import YOLO
import ultralytics

print(f"Ultralytics version: {ultralytics.__version__}")

# Test YOLOv12m-seg availability
MODEL_NAME = None
for model_option in ['yolov12m-seg.pt', 'yolo12m-seg.pt', 'yolo11m-seg.pt']:
    try:
        test_model = YOLO(model_option)
        MODEL_NAME = model_option
        print(f"✓ {model_option} loaded successfully!")
        del test_model
        break
    except Exception as e:
        print(f"⚠ {model_option} not available: {str(e)[:50]}...")

if MODEL_NAME is None:
    raise RuntimeError("No segmentation model available!")

print(f"\n🎯 Selected model: {MODEL_NAME}")

## 2. Dataset Acquisition

Download all datasets from Roboflow with instance segmentation annotations.

In [ ]:
from roboflow import Roboflow
from pathlib import Path
import shutil
import yaml

print("="*80)
print("DATASET ACQUISITION")
print("="*80)

DATASET_ROOT = Path("/content/datasets")
DATASET_ROOT.mkdir(exist_ok=True)

# Roboflow API key
ROBOFLOW_API_KEY = "nwjLvwd73Cvdh2afvIVD"
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Dataset configurations - prioritized by quality and size
datasets_config = [
    # PRIMARY: Large instance segmentation datasets
    {
        "name": "solar_panels_seg_rut",
        "workspace": "rzeszow-university-of-technology-m5ydx",
        "project": "solar-panels-seg",
        "version": 1,
        "format": "yolov8",
        "priority": 1,
        "description": "4k+ images - Rzeszow University Instance Seg"
    },
    {
        "name": "nl_solar_panel_seg",
        "workspace": "rug-uofl3",
        "project": "nl-solar-panel-seg",
        "version": 1,
        "format": "yolov8",
        "priority": 2,
        "description": "4k+ Dutch aerial imagery"
    },
    # SECONDARY: High-quality smaller datasets
    {
        "name": "solar_panels_polygons",
        "workspace": "sophia-tierney",
        "project": "solar-panels-polygons",
        "version": 1,
        "format": "yolov8",
        "priority": 3,
        "description": "86 high-quality polygon annotations"
    },
    # TERTIARY: Original datasets (proven in Round 1)
    {
        "name": "solar_panels_rf100",
        "workspace": "roboflow-100",
        "project": "solar-panels-taxvb",
        "version": 2,
        "format": "yolov8",
        "priority": 4,
        "description": "Roboflow 100 benchmark"
    },
    {
        "name": "solar_pv_detection",
        "workspace": "whereareyousolarpanel",
        "project": "solar-pv-panel-detection",
        "version": 5,
        "format": "yolov8",
        "priority": 5,
        "description": "Solar PV Panel Detection"
    },
    {
        "name": "ww_solar_panel",
        "workspace": "solar-panel-2d0l1",
        "project": "ww-solar-panel",
        "version": 16,
        "format": "yolov8",
        "priority": 6,
        "description": "WW Solar Panel global coverage"
    },
]

downloaded_datasets = []

for ds in sorted(datasets_config, key=lambda x: x['priority']):
    print(f"\n📥 [{ds['priority']}] Downloading: {ds['name']}")
    print(f"   {ds['description']}")
    
    try:
        project = rf.workspace(ds['workspace']).project(ds['project'])
        dataset = project.version(ds['version']).download(
            ds['format'], 
            location=str(DATASET_ROOT / ds['name'])
        )
        downloaded_datasets.append(ds)
        print(f"   ✓ Downloaded successfully")
    except Exception as e:
        print(f"   ⚠ Failed: {str(e)[:80]}")

print(f"\n✓ Downloaded {len(downloaded_datasets)}/{len(datasets_config)} datasets")

## 3. Dataset Merging & Preprocessing

Merge all datasets with:
- Unique prefixes to avoid filename collisions
- Normalized labels (all class 0 = solar_panel)
- Validated polygon annotations
- Proper train/val/test splits

In [ ]:
print("="*80)
print("DATASET MERGING")
print("="*80)

COMBINED_DIR = DATASET_ROOT / "combined_solar_seg"

# Create directory structure
for split in ['train', 'valid', 'test']:
    (COMBINED_DIR / split / 'images').mkdir(parents=True, exist_ok=True)
    (COMBINED_DIR / split / 'labels').mkdir(parents=True, exist_ok=True)

def normalize_and_copy_dataset(source_dir, prefix, stats):
    """
    Copy dataset with prefix, normalize labels to class 0.
    Handles both detection (5 values) and segmentation (polygon) formats.
    """
    source = Path(source_dir)
    
    for split in ['train', 'valid', 'test', 'val']:
        img_src = source / split / 'images'
        lbl_src = source / split / 'labels'
        
        if not img_src.exists():
            continue
        
        # Map 'val' to 'valid'
        target_split = 'valid' if split == 'val' else split
        img_dst = COMBINED_DIR / target_split / 'images'
        lbl_dst = COMBINED_DIR / target_split / 'labels'
        
        # Copy images
        for img_file in img_src.glob('*'):
            if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp', '.webp']:
                new_name = f"{prefix}_{img_file.name}"
                shutil.copy(img_file, img_dst / new_name)
                stats['images'] += 1
        
        # Copy and normalize labels
        if lbl_src.exists():
            for lbl_file in lbl_src.glob('*.txt'):
                new_name = f"{prefix}_{lbl_file.name}"
                
                with open(lbl_file, 'r') as f:
                    lines = f.readlines()
                
                normalized_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:  # Valid annotation
                        parts[0] = '0'  # Normalize to class 0
                        normalized_lines.append(' '.join(parts))
                        
                        # Count polygon vs bbox
                        if len(parts) > 5:
                            stats['polygons'] += 1
                        else:
                            stats['boxes'] += 1
                
                if normalized_lines:
                    with open(lbl_dst / new_name, 'w') as f:
                        f.write('\n'.join(normalized_lines) + '\n')
                    stats['labels'] += 1

# Merge all datasets
total_stats = {'images': 0, 'labels': 0, 'polygons': 0, 'boxes': 0}

for ds in downloaded_datasets:
    ds_path = DATASET_ROOT / ds['name']
    if ds_path.exists():
        prefix = ds['name'][:8]  # Short prefix
        print(f"\n📁 Merging: {ds['name']}")
        
        ds_stats = {'images': 0, 'labels': 0, 'polygons': 0, 'boxes': 0}
        normalize_and_copy_dataset(ds_path, prefix, ds_stats)
        
        print(f"   Images: {ds_stats['images']}")
        print(f"   Labels: {ds_stats['labels']}")
        print(f"   Polygons: {ds_stats['polygons']} | Boxes: {ds_stats['boxes']}")
        
        for k in total_stats:
            total_stats[k] += ds_stats[k]

print("\n" + "="*80)
print("COMBINED DATASET STATISTICS")
print("="*80)
print(f"Total Images: {total_stats['images']}")
print(f"Total Labels: {total_stats['labels']}")
print(f"Polygon Annotations: {total_stats['polygons']}")
print(f"Box Annotations: {total_stats['boxes']}")
print(f"\nSegmentation Ratio: {total_stats['polygons']/(total_stats['polygons']+total_stats['boxes']+1e-10)*100:.1f}%")

In [ ]:
# Create data.yaml
data_yaml = {
    'path': str(COMBINED_DIR),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': {
        0: 'solar_panel'
    },
    'nc': 1
}

yaml_path = COMBINED_DIR / 'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"✓ Created data.yaml at {yaml_path}")

# Count final splits
print("\n📊 Final Split Distribution:")
for split in ['train', 'valid', 'test']:
    img_path = COMBINED_DIR / split / 'images'
    lbl_path = COMBINED_DIR / split / 'labels'
    if img_path.exists():
        img_count = len(list(img_path.glob('*')))
        lbl_count = len(list(lbl_path.glob('*.txt')))
        print(f"   {split:6s}: {img_count:5d} images, {lbl_count:5d} labels")

## 4. Training Configuration

Optimized for ~4 hours on H200 with maximum quality.

In [ ]:
from ultralytics import YOLO
from datetime import datetime

print("="*80)
print("TRAINING CONFIGURATION")
print("="*80)

# Load model
model = YOLO(MODEL_NAME)
print(f"✓ Loaded {MODEL_NAME}")

# Calculate optimal batch size
if torch.cuda.is_available():
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    if gpu_mem > 70:      # H200/H100 (80GB)
        BATCH_SIZE = 32
    elif gpu_mem > 40:    # A100 (40/80GB)
        BATCH_SIZE = 24
    elif gpu_mem > 20:    # A10G/V100 (24/32GB)
        BATCH_SIZE = 16
    else:                 # Smaller GPUs
        BATCH_SIZE = 8
else:
    BATCH_SIZE = 4

print(f"GPU Memory: {gpu_mem:.0f}GB → Batch Size: {BATCH_SIZE}")

# Training configuration
training_args = {
    # ===== DATA =====
    'data': str(yaml_path),
    'imgsz': 640,
    
    # ===== TRAINING SCHEDULE =====
    'epochs': 120,              # ~4 hours on H200 with 10k images
    'batch': BATCH_SIZE,
    'patience': 25,             # Early stopping
    'close_mosaic': 15,         # Disable mosaic for last 15 epochs
    
    # ===== MODEL =====
    'task': 'segment',
    'single_cls': True,
    'pretrained': True,
    
    # ===== OPTIMIZER =====
    'optimizer': 'AdamW',
    'lr0': 0.0005,              # Lower for stability
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 5.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    
    # ===== LOSS WEIGHTS =====
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    
    # ===== AUGMENTATION (Enhanced) =====
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 15.0,            # Rotation for varied orientations
    'translate': 0.15,
    'scale': 0.7,               # More scale variation
    'shear': 5.0,               # Perspective distortion
    'perspective': 0.0005,
    'flipud': 0.5,              # Aerial imagery benefits from this
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.2,
    'copy_paste': 0.3,          # Great for segmentation!
    
    # ===== TRAINING SETTINGS =====
    'device': 0,
    'workers': 8,
    'cache': 'ram',             # Fast training
    'amp': True,                # Mixed precision
    'cos_lr': True,
    'seed': 42,
    'deterministic': True,
    
    # ===== SAVING =====
    'project': '/content/runs/segment',
    'name': 'solar_yolov12_seg',
    'save': True,
    'save_period': 10,
    'exist_ok': True,
    'verbose': True,
    'plots': True,
}

print("\n📋 Configuration Summary:")
print(f"  Model: {MODEL_NAME}")
print(f"  Epochs: {training_args['epochs']}")
print(f"  Batch Size: {training_args['batch']}")
print(f"  Image Size: {training_args['imgsz']}")
print(f"  Augmentation: Mosaic + MixUp + CopyPaste")

## 5. Training Execution

In [ ]:
print("="*80)
print("STARTING TRAINING")
print("="*80)

start_time = datetime.now()
print(f"\n🚀 Started: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Estimated: ~4 hours on H200")
print("\n" + "-"*80 + "\n")

# Train!
results = model.train(**training_args)

end_time = datetime.now()
duration = end_time - start_time

print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)
print(f"Finished: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Duration: {duration}")

## 6. Model Evaluation

In [ ]:
print("="*80)
print("MODEL EVALUATION")
print("="*80)

# Load best model
best_path = Path('/content/runs/segment/solar_yolov12_seg/weights/best.pt')
best_model = YOLO(best_path) if best_path.exists() else model
print(f"✓ Loaded: {best_path if best_path.exists() else 'current model'}")

# Validate
print("\n📊 Running validation...")
metrics = best_model.val(data=str(yaml_path))

# Extract metrics
box_map50 = float(metrics.box.map50)
box_map = float(metrics.box.map)
precision = float(metrics.box.mp)
recall = float(metrics.box.mr)
f1_score = 2 * (precision * recall) / (precision + recall + 1e-10)

seg_map50 = float(metrics.seg.map50) if hasattr(metrics, 'seg') else 0
seg_map = float(metrics.seg.map) if hasattr(metrics, 'seg') else 0

print("\n" + "="*80)
print("RESULTS")
print("="*80)
print(f"\n📦 Detection Metrics:")
print(f"   mAP@50:     {box_map50:.4f}")
print(f"   mAP@50-95:  {box_map:.4f}")
print(f"   Precision:  {precision:.4f}")
print(f"   Recall:     {recall:.4f}")
print(f"   F1 Score:   {f1_score:.4f}")

print(f"\n🎭 Segmentation Metrics:")
print(f"   Mask mAP@50:     {seg_map50:.4f}")
print(f"   Mask mAP@50-95:  {seg_map:.4f}")

# Target check
print("\n" + "="*80)
print("TARGET ACHIEVEMENT")
print("="*80)
targets = [
    ('F1 >= 0.85', f1_score, 0.85),
    ('mAP@50 >= 0.80', box_map50, 0.80),
    ('Precision >= 0.80', precision, 0.80),
    ('Recall >= 0.80', recall, 0.80),
]

all_passed = True
for name, value, target in targets:
    passed = value >= target
    status = "✓" if passed else "✗"
    print(f"   {status} {name}: {value:.4f}")
    all_passed = all_passed and passed

if all_passed:
    print("\n🎉 ALL TARGETS MET!")
else:
    print("\n⚠ Some targets not met.")

## 7. Export Training Logs & Visualizations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

print("="*80)
print("TRAINING LOGS EXPORT")
print("="*80)

results_csv = Path('/content/runs/segment/solar_yolov12_seg/results.csv')

if results_csv.exists():
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()
    
    # Save
    df.to_csv('/content/training_logs.csv', index=False)
    print("✓ Saved: /content/training_logs.csv")
    
    # Plot
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Losses
    for ax, loss_type in zip(axes[0], ['box', 'seg', 'cls']):
        train_col = f'train/{loss_type}_loss'
        val_col = f'val/{loss_type}_loss'
        if train_col in df.columns:
            ax.plot(df['epoch'], df[train_col], label='Train')
        if val_col in df.columns:
            ax.plot(df['epoch'], df[val_col], label='Val')
        ax.set_title(f'{loss_type.upper()} Loss')
        ax.legend()
        ax.grid(True)
    
    # Metrics
    if 'metrics/precision(B)' in df.columns:
        axes[1, 0].plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
        axes[1, 0].plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
        axes[1, 0].set_title('Precision & Recall')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    if 'metrics/mAP50(B)' in df.columns:
        axes[1, 1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50')
        axes[1, 1].axhline(y=0.80, color='red', linestyle='--', label='Target')
        axes[1, 1].set_title('mAP@50')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    
    if 'metrics/precision(B)' in df.columns:
        f1 = 2 * (df['metrics/precision(B)'] * df['metrics/recall(B)']) / \
             (df['metrics/precision(B)'] + df['metrics/recall(B)'] + 1e-10)
        axes[1, 2].plot(df['epoch'], f1, label='F1', color='green')
        axes[1, 2].axhline(y=0.85, color='red', linestyle='--', label='Target')
        axes[1, 2].set_title('F1 Score')
        axes[1, 2].legend()
        axes[1, 2].grid(True)
    
    plt.tight_layout()
    plt.savefig('/content/training_curves.png', dpi=150)
    plt.show()
    print("✓ Saved: /content/training_curves.png")
else:
    print("⚠ Results CSV not found")

## 8. Inference Demo

In [ ]:
import cv2

print("="*80)
print("INFERENCE DEMO")
print("="*80)

val_images = list((COMBINED_DIR / 'valid' / 'images').glob('*'))[:6]

if val_images:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for idx, img_path in enumerate(val_images):
        results = best_model.predict(str(img_path), conf=0.25, iou=0.45)
        
        annotated = results[0].plot()
        annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        
        n = len(results[0].boxes) if results[0].boxes is not None else 0
        
        axes[idx].imshow(annotated)
        axes[idx].set_title(f"Detected: {n} panels")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('/content/inference_samples.png', dpi=150)
    plt.show()
    print("✓ Saved: /content/inference_samples.png")

## 9. Export for Production

In [ ]:
print("="*80)
print("PRODUCTION EXPORT")
print("="*80)

export_dir = Path('/content/exports')
export_dir.mkdir(exist_ok=True)

# Copy best model
best_weights = Path('/content/runs/segment/solar_yolov12_seg/weights/best.pt')
if best_weights.exists():
    export_path = export_dir / 'yolov12_seg_solar_panel.pt'
    shutil.copy(best_weights, export_path)
    size_mb = export_path.stat().st_size / 1024**2
    print(f"✓ Model: {export_path.name} ({size_mb:.1f} MB)")

# Copy artifacts
for src, dst in [
    ('/content/training_logs.csv', 'training_logs.csv'),
    ('/content/training_curves.png', 'training_curves.png'),
    ('/content/inference_samples.png', 'inference_samples.png'),
]:
    if Path(src).exists():
        shutil.copy(src, export_dir / dst)
        print(f"✓ Copied: {dst}")

print(f"\n📁 Export directory: {export_dir}")
print("\n📋 Files to download:")
for f in export_dir.iterdir():
    print(f"   - {f.name}")

print("\n" + "="*80)
print("NEXT STEPS")
print("="*80)
print("1. Download yolov12_seg_solar_panel.pt")
print("2. Replace model/best.pt in your repository")
print("3. Update src/config.py with MODEL_FILENAME")
print("4. Update src/detector.py to model_type='yolo11' or 'yolo12'")
print("5. Run the pipeline!")